In [ ]:
#————————————————————

# Name: Azure OpenAI API, Prompt Engineering, Zero-Shot, One-Shot & Few-Shot Learning

# Purpose:

# This notebook will call the Azure OpenAI model GPT 3.5 Turbo (gpt-35-turbo) and create a recipe based on a set of ingredients as an input. 
# Additionally, prompt engineering techniques using a prompt engineering framework, zero-shot, one-shot and few-shot learning 
# will guide the model to achieve the desired output.

# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch), Alex Dean (adean@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 25.05.2024
# Python Version: 3.10.4

#Additionals:

# If necessary, download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\azure-openai-lab\support\requirements\requirements.txt

#————————————————————

In [24]:
# Import Python packages
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI
import sys

In [25]:
# Load required variables from .env file.
load_dotenv(dotenv_path=Path("/workspaces/azure-openai-lab/.venv/.env")) #Error sometimes due to \ or \\. Try one or the other. "C:\\Python\\azure-openai-lab\\.venv\\.env"

# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
azure_oai_key = os.environ['AZURE_OPENAI_KEY']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

In [26]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
        azure_endpoint = azure_oai_endpoint, 
        api_key=azure_oai_key,  
        api_version= "2024-02-01"
        )


In [7]:
# Zero-Shot learning. Model has a token limit of 4096.

# Create a prompt of ingredients the model should create a recipe from
ingredients = """'veal roast', 'butter', 'oil', 'carrots', 'onions', 'parsley sprigs', 'bay leaf', 'thyme', 'salt', 'pepper', 'bacon'"""

# Send request to Azure OpenAI model
response = client.chat.completions.create(
   model="gpt-35-turbo",
   temperature=0.7,
   #max_tokens=120,
   messages=[
       {"role": "system", "content": "You are a helpful cook"},
       {"role": "user", "content": "Create a flavourful recipe using the following ingredients:" + "\n" + "---" + "\n" + ingredients}
   ]
)

result = response.choices[0].message.content
print(result + "\n")

Summary: Veal Roast with Bacon and Herb-infused Vegetables

Ingredients:
- 2-pound veal roast
- 2 tablespoons butter
- 2 tablespoons oil
- 4 carrots, peeled and cut into chunks
- 2 onions, quartered
- 4 parsley sprigs
- 1 bay leaf
- 1 teaspoon thyme leaves
- Salt and pepper, to taste
- 4 slices bacon

Instructions:
1. Preheat the oven to 325°F (165°C).

2. In a large oven-safe skillet or Dutch oven, heat the butter and oil over medium-high heat. Add the veal roast and sear it on all sides until browned. Remove the roast from the skillet and set it aside.

3. In the same skillet, add the carrots, onions, parsley sprigs, bay leaf, and thyme. Season with salt and pepper. Sauté the vegetables for about 5 minutes until they begin to soften.

4. Place the veal roast on top of the sautéed vegetables in the skillet. Lay the bacon slices over the top of the roast.

5. Cover the skillet with a lid or aluminum foil and transfer it to the preheated oven. Roast for about 1 hour or until the veal re

In [27]:
# Zero-Shot learning. Model has a token limit of 4096.

# Create advanced System prompt
systemcontent = \
"""
### INSTRUCTIONS
Persona: Act as a head chef such as Joël Robuchon who specializes in simple contemporary cuisine.
Action: Create well-thought-out and flavourful recipes from a list of ingredients implementing classic culinary techniques.
Target Audience: The recipients of these recipes are couples who want to cook a special meal at least once a week.

---

### OUTPUT FORMAT
Output only one recipe and return it as a JSON object with the following format:
{"name":"","minutes":,"tags":"[]","nutrition":"[]","n_steps":"","steps":"[]","description":"","ingredients":"[]", "n_ingredients":}

The variables should contain the following information:
- name: the name of the recipe.
- minutes: the time in minutes to prepare the recipe.
- tags: a list of words that characterize the recipe.
- nutrition: a list of numeric values representing calories, total fat, sugar, sodium, protein, saturated fat, and carbohydrates.
- n_steps: the number of steps to prepare the recipe.
- steps: a list of steps to prepare the recipe.
- description: a summary of the recipe.
- ingredients: a list of the ingredient names in the recipe.
- n_ingredients: the total number of ingredients used in the recipe.
"""

# Create a prompt of ingredients the model should create a recipe from
ingredients = """'veal roast', 'butter', 'oil', 'carrots', 'onions', 'parsley sprigs', 'bay leaf', 'thyme', 'salt', 'pepper', 'bacon'"""

# Send request to Azure OpenAI model
response = client.chat.completions.create(
   model="gpt-35-turbo",
   temperature=0.7,
   #max_tokens=120,
   messages=[
       {"role": "system", "content": systemcontent},
       {"role": "user", "content": "Create a flavourful recipe using the following ingredients:" + "\n" + "---" + "\n" + ingredients + "\n" + "---"}
   ]
)

result = response.choices[0].message.content
print(result + "\n")

To create a delectable veal roast with bacon and vegetables, we will use classic culinary techniques to enhance the flavors of the ingredients. This recipe will be perfect for a special meal for couples who appreciate a well-prepared and flavorful dish.

```json
{
  "name": "Bacon-Wrapped Veal Roast with Glazed Vegetables",
  "minutes": 90,
  "tags": ["Roast", "Veal", "Bacon", "Glazed Vegetables"],
  "nutrition": [450, 25, 6, 800, 40, 10, 20],
  "n_steps": 5,
  "steps": [
    "Preheat the oven to 350°F (175°C).",
    "Season the veal roast with salt and pepper. Wrap it with bacon slices, securing them with kitchen twine.",
    "In a large skillet, heat butter and oil over medium-high heat. Sear the veal roast on all sides until browned. Remove from the skillet and set aside.",
    "In the same skillet, add carrots, onions, parsley sprigs, bay leaf, and thyme. Sauté for 5 minutes. Return the veal roast to the skillet, cover, and place in the preheated oven. Roast for 60-75 minutes or un

In [29]:
# One-Shot learning. Model has a token limit of 4096.

# Create a prompt of ingredients the model should create a recipe from
ingredients = """'veal roast', 'butter', 'oil', 'carrots', 'onions', 'parsley sprigs', 'bay leaf', 'thyme', 'salt', 'pepper', 'bacon'"""

# Create advanced System prompt
systemcontent = \
"""
### INSTRUCTIONS
Persona: Act as a head chef such as Joël Robuchon who specializes in simple contemporary cuisine.
Action: Create well-thought-out and flavourful recipes from a list of ingredients implementing classic culinary techniques.
Target Audience: The recipients of these recipes are couples who want to cook a special meal at least once a week.

---

### OUTPUT FORMAT
Output only one recipe and return it as a JSON object with the following format:
{"name":"","minutes":,"tags":"[]","nutrition":"[]","n_steps":"","steps":"[]","description":"","ingredients":"[]", "n_ingredients":}

The variables should contain the following information:
- name: the name of the recipe.
- minutes: the time in minutes to prepare the recipe.
- tags: a list of words that characterize the recipe.
- nutrition: a list of numeric values representing calories, total fat, sugar, sodium, protein, saturated fat, and carbohydrates.
- n_steps: the number of steps to prepare the recipe.
- steps: a list of steps to prepare the recipe.
- description: a summary of the recipe.
- ingredients: a list of the ingredient names in the recipe.
- n_ingredients: the total number of ingredients used in the recipe.
"""

# One-Shot learning example
ingredients_1 = """'pork spareribs', 'soy sauce', 'fresh garlic', 'fresh ginger', 'chili powder', 'fresh coarse ground black pepper', 'salt', 'fresh cilantro leaves', 'tomato sauce', 'brown sugar', 'yellow onion', 'white vinegar', 'honey', 'a.1. original sauce', 'liquid smoke', 'cracked black pepper', 'cumin', 'dry mustard', 'cinnamon sticks', 'orange, juice of', 'mirin', 'water'"""
assistant_content_1 = """{"name":"backyard style  barbecued ribs","minutes":120,"tags":"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'south-west-pacific', 'main-dish', 'pork', 'oven', 'holiday-event', 'stove-top', 'hawaiian', 'spicy', 'copycat', 'independence-day', 'meat', 'pork-ribs', 'super-bowl', 'novelty', 'taste-mood', 'savory', 'sweet', 'equipment', '4-hours-or-less']","nutrition":"[1109.5, 83.0, 378.0, 275.0, 96.0, 86.0, 36.0]","n_steps":10,"steps":"['in a medium saucepan combine all the ingredients for sauce#1 , bring to a full rolling boil , reduce heat to medium low and simmer for 1 hour , stirring often', 'rub the ribs with soy sauce , garlic , ginger , chili powder , pepper , salt and chopped cilantro , both sides !', 'wrap ribs in heavy duty foil', 'let stand 1 hour', 'preheat oven to 350 degrees', 'place ribs in oven for 1 hour , turning once after 30 minutes', '3 times during cooking the ribs open foil wrap and drizzle ribs with sauce#1', 'place all the ingredients for sauce#2 in a glass or plastic bowl , whisk well and set aside', 'remove ribs from oven and place on serving platter', 'offer both sauces at table to drizzle over ribs']","description":"this recipe is posted by request and was originaly from chef sam choy's cookbook ","ingredients":"['pork spareribs', 'soy sauce', 'fresh garlic', 'fresh ginger', 'chili powder', 'fresh coarse ground black pepper', 'salt', 'fresh cilantro leaves', 'tomato sauce', 'brown sugar', 'yellow onion', 'white vinegar', 'honey', 'a.1. original sauce', 'liquid smoke', 'cracked black pepper', 'cumin', 'dry mustard', 'cinnamon sticks', 'orange, juice of', 'mirin', 'water']","n_ingredients":22}"""

# Send request to Azure OpenAI model
response = client.chat.completions.create(
   model="gpt-35-turbo",
   temperature=0.7,
   #max_tokens=120,
   messages=[
       {"role": "system", "content": systemcontent},
       {"role": "user", "content": "Create a flavourful recipe using the following ingredients:" + "\n" + "---" + "\n" + ingredients + "\n" + "---"},
       {"role": "assistant", "content": assistant_content_1},
       {"role": "user", "content": ingredients}
   ]
)

result = response.choices[0].message.content
print(result + "\n")

```json
{
  "name": "Herb-Crusted Veal Roast with Bacon-Wrapped Vegetables",
  "minutes": 90,
  "tags": ["special-occasion", "date-night", "fancy", "roast", "herb-crusted", "bacon-wrapped", "elegant"],
  "nutrition": [450, 22, 8, 600, 40, 10, 20],
  "n_steps": 6,
  "steps": [
    "Preheat the oven to 375°F (190°C).",
    "Season the veal roast with salt and pepper, then coat it with a mixture of chopped parsley, thyme, and a hint of lemon zest.",
    "In a large skillet, heat butter and oil over medium-high heat. Sear the veal roast on all sides until browned.",
    "Place the veal roast in a roasting pan and add carrots, onions, and bay leaf around it. Wrap the vegetables with bacon slices.",
    "Roast in the oven for 60-70 minutes, or until the veal reaches an internal temperature of 145°F (63°C) for medium-rare.",
    "Allow the roast to rest for 10 minutes before slicing. Serve with the bacon-wrapped vegetables and pan juices."
  ],
  "description": "This herb-crusted veal roast i

In [31]:
# Few-Shot learning. Model has a token limit of 4096.

# Create a prompt of ingredients the model should create a recipe from
ingredients = """'veal roast', 'butter', 'oil', 'carrots', 'onions', 'parsley sprigs', 'bay leaf', 'thyme', 'salt', 'pepper', 'bacon'"""

# Create advanced System prompt
systemcontent = \
"""
### INSTRUCTIONS
Persona: Act as a head chef such as Joël Robuchon who specializes in simple contemporary cuisine.
Action: Create well-thought-out and flavourful recipes from a list of ingredients implementing classic culinary techniques.
Target Audience: The recipients of these recipes are couples who want to cook a special meal at least once a week.

---

### OUTPUT FORMAT
Output only one recipe and return it as a JSON object with the following format:
{"name":"","minutes":,"tags":"[]","nutrition":"[]","n_steps":"","steps":"[]","description":"","ingredients":"[]", "n_ingredients":}

The variables should contain the following information:
- name: the name of the recipe.
- minutes: the time in minutes to prepare the recipe.
- tags: a list of words that characterize the recipe.
- nutrition: a list of numeric values representing calories, total fat, sugar, sodium, protein, saturated fat, and carbohydrates.
- n_steps: the number of steps to prepare the recipe.
- steps: a list of steps to prepare the recipe.
- description: a summary of the recipe.
- ingredients: a list of the ingredient names in the recipe.
- n_ingredients: the total number of ingredients used in the recipe.
"""

# Few-Shot learning examples

ingredients_1 = """'pork spareribs', 'soy sauce', 'fresh garlic', 'fresh ginger', 'chili powder', 'fresh coarse ground black pepper', 'salt', 'fresh cilantro leaves', 'tomato sauce', 'brown sugar', 'yellow onion', 'white vinegar', 'honey', 'a.1. original sauce', 'liquid smoke', 'cracked black pepper', 'cumin', 'dry mustard', 'cinnamon sticks', 'orange, juice of', 'mirin', 'water'"""
assistant_content_1 = """{"name":"backyard style  barbecued ribs","minutes":120,"tags":"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'south-west-pacific', 'main-dish', 'pork', 'oven', 'holiday-event', 'stove-top', 'hawaiian', 'spicy', 'copycat', 'independence-day', 'meat', 'pork-ribs', 'super-bowl', 'novelty', 'taste-mood', 'savory', 'sweet', 'equipment', '4-hours-or-less']","nutrition":"[1109.5, 83.0, 378.0, 275.0, 96.0, 86.0, 36.0]","n_steps":10,"steps":"['in a medium saucepan combine all the ingredients for sauce#1 , bring to a full rolling boil , reduce heat to medium low and simmer for 1 hour , stirring often', 'rub the ribs with soy sauce , garlic , ginger , chili powder , pepper , salt and chopped cilantro , both sides !', 'wrap ribs in heavy duty foil', 'let stand 1 hour', 'preheat oven to 350 degrees', 'place ribs in oven for 1 hour , turning once after 30 minutes', '3 times during cooking the ribs open foil wrap and drizzle ribs with sauce#1', 'place all the ingredients for sauce#2 in a glass or plastic bowl , whisk well and set aside', 'remove ribs from oven and place on serving platter', 'offer both sauces at table to drizzle over ribs']","description":"this recipe is posted by request and was originaly from chef sam choy's cookbook ","ingredients":"['pork spareribs', 'soy sauce', 'fresh garlic', 'fresh ginger', 'chili powder', 'fresh coarse ground black pepper', 'salt', 'fresh cilantro leaves', 'tomato sauce', 'brown sugar', 'yellow onion', 'white vinegar', 'honey', 'a.1. original sauce', 'liquid smoke', 'cracked black pepper', 'cumin', 'dry mustard', 'cinnamon sticks', 'orange, juice of', 'mirin', 'water']","n_ingredients":22}"""

ingredients_2 = """'lean pork chops', 'flour', 'salt', 'dry mustard', 'garlic powder', 'oil', 'chicken rice soup'"""
assistant_content_2 = """{"name":"chicken lickin  good  pork chops","minutes":500,"tags":"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'main-dish', 'pork', 'crock-pot-slow-cooker', 'dietary', 'meat', 'pork-chops', 'equipment']","nutrition":"[105.7, 8.0, 0.0, 26.0, 5.0, 4.0, 3.0]","n_steps":5,"steps":"['dredge pork chops in mixture of flour , salt , dry mustard and garlic powder', 'brown in oil in a large skillet', 'place browned pork chops in a crock pot', 'add the can of soup , undiluted', 'cover and cook on low for 6-8 hours']","description":"here's and old standby i enjoy from time to time. it's from an old newspaper clipping i cut out years ago. very tasty.","ingredients":"['lean pork chops', 'flour', 'salt', 'dry mustard', 'garlic powder', 'oil', 'chicken rice soup']","n_ingredients":7}"""

ingredients_3 = """'boneless skinless chicken breast halves', 'condensed cream of chicken soup', 'egg', 'seasoning salt', 'all-purpose flour', 'cornstarch', 'garlic powder', 'paprika', 'salt and pepper', 'oil'"""
assistant_content_3 = """{"name":"crispy crunchy  chicken","minutes":35,"tags":"['60-minutes-or-less', 'time-to-make', 'course', 'preparation', 'healthy', 'main-dish', 'dietary', 'low-saturated-fat', 'low-in-something']","nutrition":"[335.8, 11.0, 2.0, 24.0, 64.0, 10.0, 10.0]","n_steps":8,"steps":"['combine soup , egg and seasoned salt in a bowl and set aside', 'mix together flour , cornstarch , garlic powder , paprika , salt and pepper in a resealable plastic bag', 'dip chicken pieces into soup mixture and turn so as to coat all over', 'place chicken pieces in bag with flour mixture , seal bag and shake to coat chicken', 'place coated pieces of chicken on a platter and allow to set until the coating becomes doughy', 'heat oil in a deep fryer or in a skillet over medium heat , using enough oil to cover chicken pieces when fried', 'once chicken is doughy , fry pieces in oil for approx 5-8 minutes or until cooked through and juices run clear', 'drain pieces on paper towel and serve']","description":"delicious, crunchy fried chicken. this recipe came from the ","ingredients":"['boneless skinless chicken breast halves', 'condensed cream of chicken soup', 'egg', 'seasoning salt', 'all-purpose flour', 'cornstarch', 'garlic powder', 'paprika', 'salt and pepper', 'oil']","n_ingredients":10}"""

# Send request to Azure OpenAI model
response = client.chat.completions.create(
   model="gpt-35-turbo",
   temperature=0.7,
   #max_tokens=120,
   messages=[
       {"role": "system", "content": systemcontent},
       {"role": "user", "content": "Create a flavourful recipe using the following ingredients:" + "\n" + "---" + "\n" + ingredients_1 + "\n" + "---"},
       {"role": "assistant", "content": assistant_content_1},
       {"role": "user", "content": "Create a flavourful recipe using the following ingredients:" + "\n" + "---" + "\n" + ingredients_2 + "\n" + "---"},
       {"role": "assistant", "content": assistant_content_2},
       {"role": "user", "content": "Create a flavourful recipe using the following ingredients:" + "\n" + "---" + "\n" + ingredients_3 + "\n" + "---"},
       {"role": "assistant", "content": assistant_content_3},
       {"role": "user", "content": ingredients}
   ]
)

result = response.choices[0].message.content
print(result + "\n")

{"name":"Classic Veal Roast with Bacon","minutes":180,"tags":["special-occasion", "roast", "main-course", "french-cuisine", "elegant"],"nutrition":[320, 18, 2, 700, 36, 10, 8],"n_steps":5,"steps":["Preheat the oven to 350°F (175°C).","Season the veal roast with salt and pepper.","In a Dutch oven, heat the butter and oil over medium-high heat. Sear the veal roast on all sides until golden brown.","Add carrots, onions, parsley sprigs, bay leaf, and thyme to the pot. Lay bacon slices over the top of the roast.","Cover and roast in the preheated oven for 2 to 2.5 hours, or until the veal is tender. Baste the roast with the pan juices occasionally."],"description":"This classic veal roast is a perfect dish for a special occasion. The combination of tender veal, aromatic herbs, and savory bacon creates a luxurious and flavorful main course.","ingredients":["veal roast", "butter", "oil", "carrots", "onions", "parsley sprigs", "bay leaf", "thyme", "salt", "pepper", "bacon"],"n_ingredients":11}

In [ ]:
#Transform output to pandas dataframe and save as CSV file

# Clean up Azure OpenAI Output
json_data = result.strip('` \n')

if json_data.startswith('json'):
    json_data = json_data[4:]  # Remove the first 4 characters 'json'

recipes_from_json = json.loads(json_data)
recipes_generated = pd.json_normalize(recipes_from_json)
# path_output = r"/workspaces/azure-openai-lab/data/generated_output/recipes-generated.csv" #r"C:\Python\azure-openai-lab\data\generated_output\recipes-generated.csv"
# recipes_generated.to_csv(path_output, sep='\t', encoding='utf-8', index=False)